In [1]:
# make sure that rpy2 is installed: https://rpy2.bitbucket.io/
%load_ext rpy2.ipython

import warnings
from rpy2.rinterface import RRuntimeWarning
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

# ignore R warning messages
warnings.filterwarnings("ignore", category=RRuntimeWarning)
import anndata

/home/wenhao/miniconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
%%R -o N1_DAT  -o geneinfo_N1 -o cellinfo_N1 

load('/home/wenhao/RAW_REAL/RAW_REAL/UMILOAD_8640_V2.RData')

cellinfo_N1<-data.frame(Cell=colnames(Torre_drop_sub))
geneinfo_N1<-data.frame(Gene=rownames(Torre_drop_sub))

N1_DAT<-as.data.frame(t(Torre_drop_sub))


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy.api as sc
import DCA.api as dca

Using TensorFlow backend.


In [5]:
adata_N1 = sc.AnnData(N1_DAT.values,var=geneinfo_N1,obs=cellinfo_N1)
adata_N1.var_names = geneinfo_N1.Gene
adata_N1.obs_names = cellinfo_N1.Cell
adata_N1


AnnData object with n_obs × n_vars = 1134 × 9289 
    obs: 'Cell'
    var: 'Gene'

In [6]:
%%time

res_N1 = dca.autoencode(adata_N1, training_kwargs={'verbose': False})


### Autoencoder: Successfully preprocessed 9289 genes and 1134 cells.
Calculating low dimensional representations...
Calculating reconstructions...
CPU times: user 6min 20s, sys: 3min 45s, total: 10min 5s
Wall time: 1min 53s


In [7]:
N1= res_N1.obsm['X_dca']
nr,nc = N1.shape
Br = ro.r.matrix(N1, nrow=nr, ncol=nc)
ro.r.assign("DCA_Torre", Br)

0.452157,0.255030,0.226353,0.400834,...,0.197419,0.247332,0.084920,0.209415


In [8]:
%%R
DCA_Torre<-t(DCA_Torre)
rownames(DCA_Torre)<-geneinfo_N1[,1]
colnames(DCA_Torre)<-cellinfo_N1[,1]

save(DCA_Torre,file='/home/wenhao/RAW_REAL/DCA_norm/DCA_Torre.RData')